# Uczenie maszynowe
## Implementacja wielowarstwowej sieci neuronowej
Wczytaj zbiór danych o irysach.

In [ ]:
from sklearn.datasets import load_iris
import numpy as np

iris = load_iris()
X = iris.data
y = iris.target

Twoim zadaniem będzie implementacja wielowarstwowej sieci neuronowej aktualizowanej algorymem stochastycznego spadku wzdłuż gradientu. Implementacja będzie umożliwiała dowolną architekturę warstwową.

Kluczowym elementem naszej implementacji jest ogólna klasa `Neuron` z której dziedziczyć będą wszystkie pozostałe obiekty implementujące neurony np. liniowy, ReLU, softmax itd. Nazwa `Neuron` może być trochę myląca ponieważ w rzeczywistości klasa będze implementowała całą warstwę danego typu. 

In [ ]:
class Neuron(object):
    def forward(self, x):
        return x
    
    def update(self, gradient, alpha):
        return gradient

Klasa ma dwie metody:
- `forward (x)` zwraca wynik warstwy dla podanego wejścia 
- `update (gradient, alpha)` funkcja będąca cześcią algorytmu propagacji wstecznej, wykonująca krok algorytmu SGD dla tej konkretnej warstwy. <br>  Funkcja oblicza swój lokalny gradient a następnie korzystając z reguły łańcuchowej (oraz gradientu z kolejnej warstwy który jest na wejściu funkcji) oblicza i zwraca swój gradient. W międzyczasie funkcja aktualizuje parametry warstwy (jeśli je ma) wykonując krok algorytmu SGD z szybkością optymalizacji $\alpha$.

Poniżej możesz prześledzić implementację przykładowego neuronu typu ReLU.


In [ ]:
class RELU(Neuron):
    def __init__(self):
        self.sm = None
        
    def forward(self,x):
        self.sm = np.maximum(x, 0)
        return self.sm
    
    def update(self, gradient, alpha):
        return gradient * (self.sm > 0).astype(int)

Funkcja `forward` oblicza funkcję aktywacji:
$$\max(x,0)$$
Gdyby wejście było jednowymiarowe to pochodna z tej funkcji to $1$ jeżeli $x>0$ oraz $0$ jeżeli $x<0$. Skoro wejście jest wielowymiarowe to gradient to wektor zer i jedynek dla kolejnych elementów wektora wejściowego $x$. Ponieważ w momencie wstecznej propagacji nie mamu już dostępu do $x$ to funkcja `forward` zapisuje swoje wyjście, aby je później wykorzystać w `update`.

Funkcja `update` oblicza swój lokalny gradient (wcześniej opisany, zera i jedynki) oraz przemnaża go (zgodnie z regułą łańcuchową) przez gradient wejściowy. Ponieważ sama nieliniowość nie ma żadnych parametrów to oprócz przekazania obliczonego gradientu funkcji nie robi poza tym nic (normalnie jeszcze by zaktualizowała swoje parametry).

Zwróć uwagę, że zwykle przez warstwe ReLU mamy na myśli funkcję liniową + funkcję aktywacji ReLU. Jednak skoro będziemy tak czy tak implementować wektor liniowy to nie ma potrzeby komplikowania naszego obiektu. Po prostu aby uzyskać warstwę ReLU wstawimy do sieci warstwę liniową a potem warstwę funkcji aktywacji ReLU (której implementacje właśnie widziałeś) uzyskując ten sam efekt.

Zaimplementuj warstwę (tylko aktywacja, bez liniowości) funkcji logistycznej.

In [ ]:
class Logistic(Neuron):
    def __init__(self):
        #TWÓJ KOD TUTAJ
        pass
        
    def forward(self,x):
        #TWÓJ KOD TUTAJ
        pass
    
    def update(self, gradient, alpha):
        #TWÓJ KOD TUTAJ
        pass

In [ ]:
log = Logistic()
#Poniższe wywołanie powinno zwrócić
#array([0.04742587, 0.5       , 0.73105858, 0.95257413])
log.forward(np.array([-3,0,1,3]))
#Poniższe wywołanie powinno zwrócić
#array([ 0.04517666,  0.        , -0.19661193,  0.02258833])
log.update(np.array([1,0,-1, 0.5]), 0.01)

Sercem naszej implementacji sieci warstwowej jest warstwa liniowa. Będzie to trudniejsza warstwa w implementacji bo ma ona parametry (wagi) które należy zaktualizować w czasie wstecznej propagacji.

In [ ]:
class Linear(Neuron):
    def __init__(self, n_neurons, n_inputs):
        """
        Tworzy warstwę liniową z "n_neurons" neuronami oraz "n_inputs" wejściami
        czyli wejściem do forward będzie wektor o długości n_inputs a wyjściem
        wektor o długości n_neurons.
        Pamiętaj, że wagi na początku uczenia powinny być losowe
        """
        #TWÓJ KOD TUTAJ
        pass
        
       
    def forward(self,x):
        #TWÓJ KOD TUTAJ
        pass
    
    def update(self,gradient , alpha):
        #TWÓJ KOD TUTAJ
        pass
        

In [ ]:
lin = Linear(2,3)
#Zakładając na potrzeby testów, że wagi pierszego neuronu to
# 1, 2, 3 oraz zerowy bias
# a drugiego neuronu
# 0, 1, -1 oraz zerowy bias
lin.W = np.array([[1,2,3.], [0,1,-1]])
# Poniższe wywołanie powinno zwrócić
# [ 8. -1.]
print(lin.forward(np.array([0., 1., 2.])))
# Poniższe wywołanie powinno zwrócić
# [-0.5 -0.8 -1.7]
print(lin.update(np.array([-0.5, 0.2]), 0.1))
# A po tych operacjach nowe wagi powinny wyglądać następująco:
# 1.    2.05  3.1 oraz bias równy 0.05
# 0.    0.98 -1.04 oraz bias równy -0.02
print(lin.W)
print(lin.b)

Ostatnia warstwa do implementacji to wyjściowa warstwa softmax.

In [ ]:
class SoftMax(Neuron):
    def __init__(self):
        #TWÓJ KOD TUTAJ
        pass
        
    def forward(self,x):
        #TWÓJ KOD TUTAJ
        pass
    
    def update(self, gradient, alpha):
        #TWÓJ KOD TUTAJ
        pass

Kolejnym ważnym elementem jest klasa `NeuralNetwork`, która implementuje nasz klasyfikator.

In [ ]:
class NeuralNetwork:
    def __init__(self, architecture):
        """
        Jako parametr wejściowy podajemy listę warstw sieci neuronowej 
        (czyli listę obiektów typu Neuron)
        """
        self.network = architecture
        
    def predict(self, x):
        """
        Funkcja zwraca wyjście NN dla podanego przykładu wejściowego.
        Innymi słowy funkcja wykonuje fazę forward propagation
        """
        #TWÓJ KOD TUTAJ
        pass
    
    def calculate_loss(self, X, y):
        """
        Dla podanego zbioru danych funkcja wypisuje na ekran wartości:
         - trafności klasyfikacji
         - funkcji straty: tutaj entropii krzyżowej
        Oczywiście funkcja powinna korzystać z predict
        """
        loss = None
        accuracy = None
        #TWÓJ KOD TUTAJ

        print('Acc', accuracy, 'Loss:', loss)
        
    def fit(self, X, y, alpha = 0.003, epoch = 100):
        """
        Funkcja trenująca sieć neuronową
        - alpha to szybkość uczenia
        - epoch to liczba epok do wykonania
        - X, y to zbiór uczący
        """
        for i in range(epoch): #Powtórz ile jest epok
            for j in range(X.shape[0]): #Dla każdego indeksu przykładu
                z = self.predict(X[j]) #Faza forward
                # TWÓJ KOD TUTAJ
                # 1) Gradient funkcji celu

                # 2) Faza backpropagation

            self.calculate_loss(X,y)



Zaimplementuj brakujące elementy klasy `NeuralNetwork` - radzę uzupełniać od górych funkcji do dolnych.

In [ ]:
"""
Przykładowa architektura sieci neuronowej: 
- warstwa liniowa 4 wejścia, 5 neuronów
- warstwa funkcji aktywacji ReLU
- warstwa liniowa 5 wejść (tyle w warstwie poprzedniej jest neuronów) i 3 wyjścia (tyle klas)
- warstwa funkcji aktywacji SoftMax
""" 
network = [Linear(5, 4), RELU(), Linear(3, 5), SoftMax()] 
nn = NeuralNetwork(network)

Niewytrenowanej sieci trafność powinna być ok. 0.33 (3 klasy) a strata mocno ujemna (np. -500, -1000) 

In [ ]:
nn.calculate_loss(X,y)

Uczenie z domyślnymi parametrami powinno powodować spadek funkcji straty i co jakiś czas wzrost trafności (do ok. $0.85$). Aby uzyskać lepszy wynik wywołaj komórkę kilka razy, ucząc sieć przez kolejne interacje.

In [ ]:
nn.fit(X,y) #Tranining

Jeśli jesteś w tym miejscu to należą ci się duże gratulacje!